# Visualización del proceso del Chatbot

En este fichero seremos capaces de entender de manera clara y detallada el proceso que sigue nuestro Chatbot desde que se realiza una pregunta hasta que se produce una respuesta.

Es importante matizar que este Notebook es un ejemplo del funcionamiento del sistema en el cuál se utiliza como fuente de conocimiento los datos introducidos por el propio usuario.

El sistema tiene la capacidad de trabajar con distintas fuentes de conocimiento, de las cuáles obtiene la información necesaria y no es necesario que esta sea introducida de forma manual. No obstante, como material docente hemos considerado mas interesante que el script de visualización funcionase así.

In [1]:
from natural_language import Analysis, compose_answer
import json

## Creación de la conversación
Comenzamos por crear un objeto Ground, el cuál será el encargado de gestionar la conversación del usuario con el sistema.
A cada una de estas conversaciones se le asigna un identificador único.

In [2]:
from ground import Ground

In [3]:
ground = Ground()
print("Creado Ground con id: {}".format(ground.ground_id))

Creado Ground con id: 0


## Creación de preguntas
A modo de testeo, vamos a probar a añadir conocimiento manualmente a esta conversación, si bien se podría haber realizado una fase de recuperación de la información a traves de cualquier tipo de base de datos.

Cada vez que proporcionemos conocimiento adicional este puede contener:
- Una pregunta.
- Respuestas ideales.
- Fragmentos de información relevante sobre la pregunta.

In [10]:
from test_cases import do_tests, response
from grafeno.jupyter import visualize

### Introducción de la pregunta

Realizamos una pregunta, la cuál será analizada y de ella se obtendrá un grafo como el que se ve a continuación.

In [5]:
ground.teardown()
introduceQuestion = str(input("Introduce question: ")) # Example: John helps Mary?
analysis_question = Analysis(introduceQuestion)
snippets = []
ideals = []
all_data = []
visualize(analysis_question.graph)

Introduce question: John helps Mary?


### Introducción del conocimiento en la conversación

Durante esta etapa, podemos añadir conocimiento a nuestra conversación que nos permitirá encontrar las respuestas a la consulta introducida anteriormente.

La siguiente celda se puede ejecutar varias veces si deseamos añadir varios fragmentos de información.

Además, como en el apartado anterior, se realiza un analisis de cada uno de estos fragmentos de información y genera un grafo como el que podemos observar a continuación.

In [11]:
snippet = str(input("Introduce snippet: ")) # Example: John helps Mary at weekends very much.
print('')
snippets.append({'text':snippet})
a = Analysis(snippet)
ground.add_text(a)
visualize(a.graph)

Introduce snippet: John helps Mary at weekends very much.

Cypher insert query: 
CREATE (n:VERB {concept: 'help', sempos: 'v', tense: 'None', polarity: '+', ground_id: '0', _temp_id: '0'});
Cypher insert query: 
CREATE (n:NOUN {concept: 'john', sempos: 'n', proper: 'False', num: 'p', ground_id: '0', _temp_id: '1'});
Cypher insert query: 
CREATE (n:NOUN {concept: 'mary', sempos: 'n', proper: 'False', num: 'p', ground_id: '0', _temp_id: '2'});
Cypher insert query: 
CREATE (n:NOUN {concept: 'weekend', sempos: 'n', proper: 'False', num: 'p', ground_id: '0', _temp_id: '3'});
Cypher insert query: 
CREATE (n:ADVERB {concept: 'much', sempos: 'r', ground_id: '0', _temp_id: '4'});
Cypher insert query: 
CREATE (n:ADVERB {concept: 'very', sempos: 'r', ground_id: '0', _temp_id: '5'});
Cypher query: 
MATCH (n {_temp_id: '0'}), (m {_temp_id: '1'}) CREATE (n)-[r:AGENT {weight: '1.0'}]->(m);
Cypher query: 
MATCH (n {_temp_id: '0'}), (m {_temp_id: '2'}) CREATE (n)-[r:THEME {weight: '1.0'}]->(m);
Cypher 

Introducimos manualmente, la respuesta o respuestas ideales a nuestra pregunta.
Esta no es una de las fases de nuestro Chatbot, pero nos servirá para más adelante verificar si la respuesta obtenida por el mismo es satisfactoria o no.

In [7]:
ideal = str(input("Introduce ideal answer: ")) # Example: Yes
ideals.append(ideal)

Introduce ideal answer: Yes.


### Resolución de la consulta y construcción de la respuesta

Realizamos finalmente el matching entre el grafo de pregunta y los grafos de conocimiento introducidos en nuestra conversación actual.

Cabe destacar que a la hora de construir dicha respuesta, esta se hace en función de si la pregunta es abierta o cerrada.

- En caso de que la pregunta sea cerrada: comprobamos el número de respuestas encontradas y contestamos Yes/No.
- En caso de que la pregunta sea abierta: devolvemos las respuestas encontradas como frases en lenguaje natural. Para ello utilizamos la función graft en la cual partiendo de la consulta y las respuestas encontradas las combinamos para componer las frases en lenguaje natural.

In [8]:
question_type = analysis_question.graph.question_type
answers = ground.ask_question(analysis_question)
print('Question: {}'.format(introduceQuestion))
print('Answer: {}'.format(response(analysis_question,answers,question_type)))
print('Ideal Answer: {}'.format(ideal))

Question: John helps Mary?
Answer: Yes
Ideal Answer: Yes.


### Distinción del conocimiento entre conversaciones

Podemos comprobar que si realizamos la misma pregunta en una conversación diferente (en este caso una nueva), no va a encontrar la respuesta al encontrarse la correspondiente base de conocimiento vacía o no contener la información necesaria para dicha pregunta.

In [11]:
ground2 = Ground()
answers2 = ground2.ask_question(a)
print('Answer: {}'.format(response(analysis_question,answers2,question_type)))

Answer: No


Por último, podemos almacenar todo este proceso en un fichero para posteriromente poder realizar de nuevo todo este análisis.

In [13]:
question = {'body':introduceQuestion,'ideal_answer':ideals, 'snippets':snippets};
all_data.append(question)
data = {'questions':all_data}
with open('Datasets/resume_test_cases-SampleData.json', 'w') as js:
    json.dump(data, js)

La celda siguiente nos servirá para cargar dicho fichero y realizar todo el análisis con los datos introducidos previamente.

In [14]:
do_tests('Datasets/resume_test_cases-SampleData.json','Datasets/resume_test_cases-Answer.json')

------------------------------------
Question: John helps Mary?
Question type: closed
Answer: Yes
